In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import pickle

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
df = pd.read_csv('dataDash.csv', encoding ='iso-8859-9')
dfPred = pd.read_csv('dfPred.csv', encoding ='iso-8859-9')
available_indicators0 = ['car', 'body', 'engV', 'engType', 'registration', 'model', 'drive']
available_indicators = ['mileage','price','year']
brand_indicators = df['car'].unique()
brand_indicators.sort()
model_indicators = list(df['model'].value_counts().keys())
model_indicators.sort()
body_indicators = list(df['body'].value_counts().keys())
body_indicators.sort()
engType_indicators = list(df['engType'].value_counts().keys())
engType_indicators.sort()
reg_indicators = list(df['registration'].value_counts().keys())
reg_indicators.sort()
drive_indicators = list(df['drive'].value_counts().keys())
drive_indicators.sort()






app.layout = html.Div(
    children=[
        html.H1('Dashboard Final Project'),
        html.Div(children='''
        Created by: Dimas
    '''),
        
        dcc.Tabs(
            children=[
                dcc.Tab(
                    value='Tab1',
                    label='Graph of Features',
                    children=[
                         html.Div([

                                html.Div([
                                    dcc.Dropdown(
                                        id='xaxis-column0',
                                        options=[{'label': i, 'value': i} for i in available_indicators0],
                                        value='car'
                                    ),
                                ],
                                style={'width': '48%', 'display': 'inline-block'}),

                            ]),

                            dcc.Graph(id='indicator-graphic0'),


                            html.Br(),
                            html.Br(),

                            html.Div([

                                    html.Div([
                                        dcc.Dropdown(
                                            id='xaxis-column',
                                            options=[{'label': i, 'value': i} for i in available_indicators],
                                            value='mileage'
                                        ),
                                        dcc.RadioItems(
                                            id='xaxis-type',
                                            options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                                            value='Linear',
                                            labelStyle={'display': 'inline-block'}
                                        )
                                    ],
                                    style={'width': '48%', 'display': 'inline-block'}),

                                    html.Div([
                                        dcc.Dropdown(
                                            id='yaxis-column',
                                            options=[{'label': i, 'value': i} for i in available_indicators],
                                            value='price'
                                        ),
                                        dcc.RadioItems(
                                            id='yaxis-type',
                                            options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                                            value='Linear',
                                            labelStyle={'display': 'inline-block'}
                                        )
                                    ],style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
                                ]),

                                dcc.Graph(id='indicator-graphic'),
                    ]),
                
                 dcc.Tab(
                    value='Tab2',
                    label='Graph of Features',
                    children=[
                            html.Br(),                          
                            dcc.Dropdown(
                                id='brand',
                                options=[{'label': i, 'value': i} for i in brand_indicators],
                                value='Volkswagen'
                            ), 
                            html.Br(),  
                            dcc.Dropdown(
                                id='body',
                                options=[{'label': i, 'value': i} for i in body_indicators],
                                value='sedan'
                            ),
                            html.Br(),  
                            dcc.Dropdown(
                                id='model',
                                options=[{'label': i, 'value': i} for i in model_indicators],
                                value='E-Class'
                            ), 
                            html.Br(),  
                            dcc.Dropdown(
                                id='engType',
                                options=[{'label': i, 'value': i} for i in engType_indicators],
                                value='Petrol'
                            ),
                            html.Br(),  
                            dcc.Dropdown(
                                id='reg',
                                options=[{'label': i, 'value': i} for i in reg_indicators],
                                value='yes'
                            ),
                            html.Br(),                          
                            dcc.Dropdown(
                                id='drive',
                                options=[{'label': i, 'value': i} for i in drive_indicators],
                                value='front'
                            ), 
                            html.Br(),  
                            dcc.Input(id='year', value='', type='number', placeholder = 'year (e.g:2002)'),  
                            html.Br(),                         
                            html.Br(),  
                            dcc.Input(id='mileage', value='', type='number', placeholder = 'mileage (e.g:2000)'),  
                            html.Br(),        
                            html.Br(),  
                            dcc.Input(id='engV', value='', type='number', placeholder = 'engV (e.g:2.1)'),                        
                            html.Br(),      
                            html.Br(),  
                            html.Button('Click Me', id='button'),
                            html.Br(),   
                            html.Br(),  
                            html.H1(id='my-div')
                    ])
            ]),
    ]    
)



@app.callback(
    Output(component_id='my-div', component_property='children'),
    [Input('button', 'n_clicks')],
    state=[State(component_id='brand', component_property='value'),          
          State(component_id='body', component_property='value'),
          State(component_id='model', component_property='value'),            
          State(component_id='engType', component_property='value'),
          State(component_id='reg', component_property='value'),
          State(component_id='drive', component_property='value'),
          State(component_id='mileage', component_property='value'),
          State(component_id='engV', component_property='value'),
          State(component_id='year', component_property='value')          ]
)

def price_predict(n_clicks, brand, body, model, engType, reg, drive, mileage, engV, year):
    brand = df[df['car'] == brand].mean()['price']
    model = df[df['model'] == model].mean()['price']
    
    body = 'body_'+body
    bdy = {'body_hatch':0,'body_other':0,'body_sedan':0,'body_vagon':0,'body_van':0}
    if body in bdy.keys():
        bdy[body] = 1
    
    engType = 'engType_'+engType
    eng = {'engType_Gas':0,'engType_Other':0,'engType_Petrol':0}
    if engType in eng.keys():
        eng[engType] = 1
    
    reg = 'registration_'+reg
    re = {'registration_yes':0}
    if reg in re.keys():
        re[reg] = 1
        
    drive = 'drive_'+drive
    drv = {'drive_full':0, 'drive_rear':0}
    if drive in drv.keys():
        drv[drive] = 1

    ml = pickle.load(open('/Users/dimas/Documents/DataScience/JC_DATA_SCIENCE/kelasModul/projectAkhir/car/priceUsedCar_gb_model.sav','rb'))
    result = str(ml.predict([[mileage,engV,year,brand,model,bdy['body_hatch'],
                              bdy['body_other'],bdy['body_sedan'],bdy['body_vagon'],
                              bdy['body_van'], eng['engType_Gas'], eng['engType_Other'],
                              eng['engType_Petrol'], re['registration_yes'],
                              drv['drive_full'], drv['drive_rear']]]))
    
    return 'Sum Insured : $ ' + str(result)[1:-1]

@app.callback(
    Output('indicator-graphic0', 'figure'),
    [Input('xaxis-column0', 'value')
    ])

def update_graphs0(xaxis_column_name):
    title1 = {
            'data': [{
                'x': df[xaxis_column_name].value_counts().iloc[:10].index,
                'y': df[xaxis_column_name].value_counts().iloc[:10].values,
                'type': 'bar',
            }],
            'layout': {             
                    'title': f'The most 10 of {xaxis_column_name} for Sale'
            }
        }
    
    title2 = {
            'data': [{
                'x': df[xaxis_column_name].value_counts().iloc[:10].index,
                'y': df[xaxis_column_name].value_counts().iloc[:10].values,
                'type': 'bar',
            }],
            'layout': {             
                    'title': f'Most preferred {xaxis_column_name} used over the years'
            }
        }

    if xaxis_column_name == 'model' or xaxis_column_name == 'engV' or xaxis_column_name == 'car':
        return title1
    else:
        return title2
            
        

@app.callback(
    Output('indicator-graphic', 'figure'),
    [Input('xaxis-column', 'value'),
     Input('yaxis-column', 'value'),
     Input('xaxis-type', 'value'),
     Input('yaxis-type', 'value'),
    ])
def update_graphs(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type
                ):
    dff = df

    return {
        'data': [dict(
            x=dff[xaxis_column_name],
            y=dff[yaxis_column_name],
            mode='markers',
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'white'}
            }
        )],
        'layout': dict(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 40, 't': 10, 'r': 0},
            hovermode='closest'
        )
    }


if __name__ == '__main__':
    app.run_server(port = 8051)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8051/ (Press CTRL+C to quit)
